In [54]:
import numpy as np 
import pandas as pd

# clean for state

Load in the data frame. Get lists of column labels.

In [55]:
all_data = pd.read_csv("Data_Files/state_housing_zillow.csv")
col_labels = all_data.columns.tolist()
timestamps = col_labels[5:]

Tranpose the dataset so that it is indexed by time and each column represents a state.

In [56]:
price_by_state_series = pd.DataFrame(all_data[timestamps].values.T, columns=all_data['RegionName'], index=timestamps)
print(price_by_state_series)

RegionName     California          Texas        Florida       New York  \
2000-01-31  186429.815199  110538.064827  105825.018501  149355.915577   
2000-02-29  187056.461453  110598.134067  106054.245342  149887.769350   
2000-03-31  187902.587345  110627.268589  106332.113966  150399.764377   
2000-04-30  189739.274366  110771.865807  106896.949408  151500.926494   
2000-05-31  191874.146347  110865.509233  107506.063565  152640.235655   
...                   ...            ...            ...            ...   
2024-09-30  768060.404666  299577.201315  389685.944614  470329.929340   
2024-10-31  769640.332853  299320.975181  388747.458761  472704.198362   
2024-11-30  771401.672972  298954.558970  387558.837887  474502.099921   
2024-12-31  773183.509222  298684.555574  386475.569667  476224.642087   
2025-01-31  773347.013436  298510.934462  385540.771612  477332.960271   

RegionName   Pennsylvania       Illinois           Ohio        Georgia  \
2000-01-31   96116.779767  125204.519

Create a dataframe of labels. Add a column to one-hot encode whether the original dataframe contained any NaN values for that state. Default to 0 (False).

In [57]:
state_labels = pd.DataFrame(all_data[col_labels[:5]].values, columns=col_labels[:5], index=all_data['RegionName'])
print(state_labels)
state_labels.drop(columns=['StateName'], inplace=True)
state_labels['LeadingNaN'] = 0
state_labels['ContainsNaN'] = 0

                     RegionID SizeRank            RegionName RegionType  \
RegionName                                                                
California                  9        0            California      state   
Texas                      54        1                 Texas      state   
Florida                    14        2               Florida      state   
New York                   43        3              New York      state   
Pennsylvania               47        4          Pennsylvania      state   
Illinois                   21        5              Illinois      state   
Ohio                       44        6                  Ohio      state   
Georgia                    16        7               Georgia      state   
North Carolina             36        8        North Carolina      state   
Michigan                   30        9              Michigan      state   
New Jersey                 40       10            New Jersey      state   
Virginia                 

Address NaN values. Replace any leading NaN values with the oldest recorded price measurement (backwards fill method). For intermediate NaN values, use take the average of the values before and after (interpolate). Update the ContainsNaN column of the labels dataframe.

In [58]:
# find columns with NaN values and update the column in labels
col_nans = price_by_state_series.isna().any()
print(np.sum(col_nans.values))
#print(col_nans)

# interpolate intermediate NaN values
for state in all_data['RegionName']:
    #print(col_nans[state])
    if col_nans[state]:
        all_nans = price_by_state_series[state].isna().sum()
        price_by_state_series[state].interpolate(method='linear', inplace=True)
        new_nans = price_by_state_series[state].isna().sum()
        state_labels.loc[state, 'ContainsNaN'] = all_nans - new_nans
        #if all_nans - new_nans != 0:
        print(state, all_nans - new_nans, all_nans, new_nans)   

print()
# backwards fill leading NaN values
col_nans = price_by_state_series.isna().any()
print(np.sum(col_nans.values))
for state in all_data['RegionName']:
    if col_nans[state]:
        print(state, price_by_state_series[state].isna().sum())
        state_labels.loc[state, 'LeadingNaN'] = price_by_state_series[state].isna().sum()
        price_by_state_series[state].fillna(method='bfill', inplace=True)


9
Arizona 1 1 0
New Mexico 0 27 27
Idaho 1 1 0
West Virginia 1 1 0
Montana 0 61 61
South Dakota 1 1 0
North Dakota 0 108 108
Alaska 1 1 0
Wyoming 0 27 27

4
New Mexico 27
Montana 61
North Dakota 108
Wyoming 27


/var/folders/0r/8vfq070d6h9_gmryx7sk_rsw0000gp/T/ipykernel_73883/1870249620.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  price_by_state_series[state].interpolate(method='linear', inplace=True)
/var/folders/0r/8vfq070d6h9_gmryx7sk_rsw0000gp/T/ipykernel_73883/1870249620.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which w

In [59]:
col_nans = price_by_state_series.isnull().any()
print(col_nans)

RegionName
California              False
Texas                   False
Florida                 False
New York                False
Pennsylvania            False
Illinois                False
Ohio                    False
Georgia                 False
North Carolina          False
Michigan                False
New Jersey              False
Virginia                False
Washington              False
Arizona                 False
Massachusetts           False
Tennessee               False
Indiana                 False
Maryland                False
Missouri                False
Wisconsin               False
Colorado                False
Minnesota               False
South Carolina          False
Alabama                 False
Louisiana               False
Kentucky                False
Oregon                  False
Oklahoma                False
Connecticut             False
Utah                    False
Iowa                    False
Nevada                  False
Arkansas                False

Export cleaned datasets.

In [60]:
price_by_state_series.to_csv("Data_Files/price_by_state_cleaned.csv")
state_labels.to_csv("Data_Files/state_data_labels.csv")

In [61]:
state_labels

,RegionID,SizeRank,RegionName,RegionType,LeadingNaN,ContainsNaN
RegionName,,,,,,
California,9,0,California,state,0,0
Texas,54,1,Texas,state,0,0
Florida,14,2,Florida,state,0,0
New York,43,3,New York,state,0,0
Pennsylvania,47,4,Pennsylvania,state,0,0
Illinois,21,5,Illinois,state,0,0
Ohio,44,6,Ohio,state,0,0
Georgia,16,7,Georgia,state,0,0
North Carolina,36,8,North Carolina,state,0,0


# clean for county

Load in the data frame. Get lists of column labels.

In [62]:
all_data = pd.read_csv("Data_Files/county_housing_zillow.csv")
col_labels = all_data.columns.tolist()
timestamps = col_labels[9:]

Tranpose the dataset so that it is indexed by time and each column represents a state.

In [63]:
price_by_county_series = pd.DataFrame(all_data[timestamps].values.T, columns=all_data['RegionName'], index=timestamps)
print(price_by_county_series)

RegionName  Los Angeles County    Cook County  Harris County  Maricopa County  \
2000-01-31       205554.642143  146088.131319  109502.880748    143224.384045   
2000-02-29       205787.076303  146058.811433  109474.767351    143516.508250   
2000-03-31       206645.170206  146299.372635  109313.922526    143902.521074   
2000-04-30       208344.164074  146991.131284  109247.781175    144696.080612   
2000-05-31       210504.871387  147876.373157  109221.904554    145570.897200   
...                        ...            ...            ...              ...   
2024-08-31       857411.008912  302846.384103  282907.172194    468743.383602   
2024-09-30       863957.219674  303511.080209  282692.273148    467798.713506   
2024-10-31       869036.446028  303864.098187  282325.752609    467070.117625   
2024-11-30       873116.490139  304254.182907  281874.816774    466405.864875   
2024-12-31       876489.981962  304914.611846  281642.532453    465828.629769   

RegionName  San Diego Count

Create a dataframe of labels. Add a column to one-hot encode whether the original dataframe contained any NaN values for that state. Default to 0 (False).

In [76]:
county_labels = pd.DataFrame(all_data[col_labels[:9]].values, columns=col_labels[:9], index=all_data['RegionName'])
county_labels['LeadingNaN'] = 0
county_labels['ContainsNaN'] = 0

Address NaN values. Replace any leading NaN values with the oldest recorded price measurement (backwards fill method). For intermediate NaN values, use take the average of the values before and after (interpolate). Update the ContainsNaN column of the labels dataframe.

In [ ]:
# find columns with NaN values and update the column in labels
col_nans = price_by_county_series.isna().any()
print(np.sum(col_nans.values))

# interpolate intermediate NaN values
print(col_nans)
for county in all_data['RegionName']:
    print(county)
    print(col_nans[county])
    if col_nans[county]:
        all_nans = price_by_county_series[county].isna().sum()
        price_by_county_series[county].interpolate(method='linear', inplace=True)
        new_nans = price_by_county_series[county].isna().sum()
        county_labels.loc[county, 'ContainsNaN'] = all_nans - new_nans
        #if all_nans - new_nans != 0:
        print(county, all_nans - new_nans, all_nans, new_nans)   

print()
# backwards fill leading NaN values
col_nans = price_by_county_series.isna().any()
print(np.sum(col_nans.values))
for county in all_data['RegionName']:
    if col_nans[county]:
        print(county, price_by_county_series[county].isna().sum())
        county_labels.loc[county, 'LeadingNaN'] = price_by_county_series[county].isna().sum()
        price_by_county_series[county].fillna(method='bfill', inplace=True)

2363
RegionName
Los Angeles County    False
Cook County           False
Harris County         False
Maricopa County        True
San Diego County      False
                      ...  
Banner County          True
Daggett County         True
Thomas County          True
McPherson County       True
Petroleum County       True
Length: 3073, dtype: bool
Los Angeles County
False
Cook County
RegionName
Cook County    False
Cook County    False
Cook County     True
dtype: bool


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
col_nans = price_by_county_series.isnull().any()
print(col_nans)

Export cleaned datasets.

In [ ]:
price_by_county_series.to_csv("Data_Files/price_by_county_cleaned.csv")
county_labels.to_csv("Data_Files/county_data_labels.csv")